# LECL인터페이스

`stream` : 응답의 chunk를 streaming한다.

`invoke` : 입력에 대해 체인을 호출한다.

`batch`: 입력 목록에 대한 체인을 호출.

### 비동기 메소드

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_teddynote import logging

logging.langsmith("CH01.Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01.Basic


In [8]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [11]:
model = ChatOllama(model="llama3:8b")
template= """
{topic}에 대하여 3문장으로 설명해줘
"""
prompt = PromptTemplate.from_template(template)

chain = prompt | model | StrOutputParser()

In [20]:
chain

PromptTemplate(input_variables=['topic'], template='\n{topic}에 대하여 3문장으로 설명해줘\n')
| ChatOllama(model='llama3:8b')
| StrOutputParser()

# stream: 실시간 출력

In [18]:
answer = ""
for token in chain.stream({'topic': "멀티모달"}):
    print(token, end="", flush=True)
    answer += token

Here are three sentences explaining multimodal:

Multimodal refers to the combination of multiple modes or forms of expression, such as text, image, audio, and video, to convey meaning and communicate information. In a multimodal system, different modalities can be used together to provide a more comprehensive and engaging experience for users. By leveraging the strengths of each modality, multimodal systems can improve comprehension, recall, and overall user satisfaction in various applications, such as education, entertainment, and social media.

In [19]:
answer

'Here are three sentences explaining multimodal:\n\nMultimodal refers to the combination of multiple modes or forms of expression, such as text, image, audio, and video, to convey meaning and communicate information. In a multimodal system, different modalities can be used together to provide a more comprehensive and engaging experience for users. By leveraging the strengths of each modality, multimodal systems can improve comprehension, recall, and overall user satisfaction in various applications, such as education, entertainment, and social media.'

# batch: 배치 단위 실행

여러개의 딕셔너리를 포함하는 리스트를 인자로 받아 각 딕셔너리에 있는 topic키의 값을 사용하여 일괄 처리를 수행

즉, 한번의 여러개를 호출할때 사용

In [21]:
answer = chain.batch([{"topic":"ChatGPT"}, {"topic": "Instagram"}])

In [24]:
print(answer[0])
print(answer[1])

Here are three sentences explaining ChatGPT:

ChatGPT is a cutting-edge language model developed by Meta AI that can understand and respond to human input in a conversational manner. It uses a massive dataset of text from the internet to learn patterns and relationships, allowing it to generate human-like responses to a wide range of questions and topics. Through its ability to process and generate natural language, ChatGPT has the potential to revolutionize fields such as customer service, content creation, and even artificial intelligence research itself.
Here are three sentences that describe Instagram:

Instagram is a social media platform where users can share photos and videos with friends, family, and followers. Users can upload content to their own feed, as well as engage with others by liking, commenting, and sharing posts. With over a billion active users, Instagram has become one of the most popular social media platforms in the world, allowing people to express themselves v

In [27]:
chain.batch(
    [
            {"topic":"ChatGPT"},
            {"topic":"Instagram"},
            {"topic":"멀티모달"},
            {"topic":"프로그래밍"},
            {"topic":"머신러닝"},
    ],
    config={"max_concurrency":3}, #최대 3개씩
)

['Here are three sentences explaining ChatGPT:\n\nChatGPT is a cutting-edge language model developed by Meta AI that can understand and respond to human input in a conversational manner. It uses a combination of machine learning algorithms and large datasets to generate human-like text that can answer questions, provide information, and even engage in creative activities like storytelling and joke-telling. By leveraging its vast knowledge base and advanced natural language processing capabilities, ChatGPT has the potential to revolutionize various industries and aspects of our daily lives, from customer service and education to entertainment and healthcare.',
 "Here are three sentences that explain Instagram:\n\nInstagram is a social media platform where users can share photos and videos with their followers, as well as edit and enhance them using various filters and effects. Founded in 2010 by Kevin Systrom and Mike Krieger, Instagram was acquired by Facebook in 2012 and has since bec

# async steam: 비동기 스트림

어싱크러너스

- 특정 task를 할 때, 이 task가 종료될 때 까지, 기다리는것은 동기 스트림이다.
- 특정 task를 할 때 다른것을 수행하는것을 비동기 스트림이다. 즉, 작읩의 완료를 기다리는 동안 다른 코드의 실행이 가능하다.

In [33]:
# #비동기 체인 객체의 "ainvoke"메서드를 출력하여 "nvda"토픽을 처리한다.
chain.invoke({"topic": "nvda"})

'Here are three sentences explaining NVDA:\n\nNVDA (NonVisual Desktop Access) is a free and open-source screen reader software that provides auditory feedback to users who are blind or have low vision. It reads aloud the text on the computer screen, including menus, buttons, and other interface elements, allowing users to navigate and interact with digital content without visual assistance. NVDA also supports braille displays and offers a range of customization options to suit individual needs and preferences.'

In [31]:
my_process = chain.ainvoke({"topic":"nvda"})

In [32]:
# 비동기로 처리되는 프로세스가 끝날떄까지 기다린다.
await my_process

"Here are three sentences explaining NVDA:\n\nNVDA (NonVisual Desktop Access) is a free and open-source screen reader software that provides auditory feedback to users who are blind or have low vision. It reads aloud the text on a computer's screen, allowing users to navigate and interact with digital content without relying on visual cues. With NVDA, users can access a wide range of applications, including web browsers, email clients, word processors, and more, using a keyboard-only interface or with assistive technologies such as braille displays or speech-output devices."